In [43]:
# Data manipulation
import numpy as np
import pandas as pd

# Plotting
import plotly.express as px
import matplotlib.pyplot as plt
import streamlit as st

import datetime

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import quant finance libraries
import yfinance as yf

In [44]:
stocks = ['eng.mc','ele.mc', 'itx.mc', 'bbva.mc', 'vid.mc', 'rep.mc', 'ibe.mc', 'or.pa',
                'san.pa', 'azn', 'regn', 'atvi', 'msft', 'team', 'googl', 'nvda', 'csx']
start_date = datetime.date(2020, 1, 1)

In [45]:
def get_data(stocks, start_date, end_date):
            return yf.download(stocks, start = start_date, end = end_date)

In [46]:
stocks

['eng.mc',
 'ele.mc',
 'itx.mc',
 'bbva.mc',
 'vid.mc',
 'rep.mc',
 'ibe.mc',
 'or.pa',
 'san.pa',
 'azn',
 'regn',
 'atvi',
 'msft',
 'team',
 'googl',
 'nvda',
 'csx']

In [47]:
start_date

datetime.date(2020, 1, 1)

In [48]:
df = get_data(stocks, start_date = "2019-01-01", end_date = "2022-11-30")       
df = df['Adj Close']

[*********************100%***********************]  17 of 17 completed


In [49]:
def cum_returns(stocks, wts):

  weighted_returns = (wts * stocks.pct_change()[1:])
  weighted_returns = pd.DataFrame(weighted_returns)
  port_ret = weighted_returns.sum(axis=1)
  return (port_ret + 1).cumprod() 

In [50]:
total_stocks = len(stocks)
weight = [1/total_stocks]*total_stocks
#call cumulative returns
returns = cum_returns(df, weight).reset_index()
returns.rename(columns={'Date':'date', 0:'ret'}, inplace=True)

In [51]:
# optimizitation
from scipy.optimize import minimize 

def optimize_weights(returns, risk_free_return):
    
    n = returns.shape[1]
    initial_weights = np.ones(n) / n
    bounds = [(0, 1) for i in range(n)]
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    def neg_sharpe_ratio(weights, returns, risk_free_rate):
        portfolio_return = np.sum(returns.mean() * weights) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        return -sharpe_ratio
    result = minimize(fun=neg_sharpe_ratio, x0=initial_weights, args=(returns, risk_free_return), method='SLSQP', bounds=bounds, constraints=constraints)
    optimized_weights = result.x
    return optimized_weights

In [61]:
res = optimize_weights(df.pct_change(), 4)

In [77]:
pd.DataFrame(data=res, index=df.columns, columns=['res']).sort_values(by='res', ascending=False)[:3].index.tolist()

['NVDA', 'SAN.PA', 'ENG.MC']

In [66]:
import tensorflow as tf 

def optimize_weights_with_adam(returns, target_return, learning_rate=0.01, epochs=1000):
  n = returns.shape[1]
  initial_weights = tf.ones((n,), dtype=tf.float32) / n
  returns_mean = tf.reduce_mean(returns, axis=0)
  returns_cov = tf.matmul(tf.transpose(returns - returns_mean), returns - returns_mean) / (returns.shape[0] - 1)

  def portfolio_return(weights, mean_returns, cov_matrix):
    portfolio_return = tf.tensordot(weights, mean_returns, axes=1)
    portfolio_volatility = tf.sqrt(tf.tensordot(tf.tensordot(weights, cov_matrix, axes=1), weights, axes=1))
    return portfolio_return, portfolio_volatility

  def loss_fn(weights, mean_returns, cov_matrix, target_return):
    portfolio_return, portfolio_volatility = portfolio_return(weights, mean_returns, cov_matrix)
    return -(portfolio_return - target_return) / portfolio_volatility

  optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

  def train_step(weights, mean_returns, cov_matrix, target_return):
    with tf.GradientTape() as tape:
      loss_value = loss_fn(weights, mean_returns, cov_matrix, target_return)
    grads = tape.gradient(loss_value, weights)
    optimizer.apply_gradients(zip([grads], [weights]))
    return weights

  weights = initial_weights
  for epoch in range(epochs):
    weights = train_step(weights, returns_mean, returns_cov, target_return)
  
  optimal_weights = weights / tf.reduce_sum(weights)
  return optimal_weights.numpy()

In [67]:
res_adam = optimize_weights(df.pct_change(), 4)

In [76]:
pd.DataFrame(data=res_adam, index=df.columns, columns=['res']).sort_values(by='res', ascending=False)[:3]

,res
NVDA,1.000000e+00
SAN.PA,6.272621e-13
ENG.MC,5.107928e-13
